In [1]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint 
from keras.models import load_model, Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPool2D, BatchNormalization, Dropout, Layer, LSTM, Input
from sklearn.metrics import mean_absolute_error

In [2]:
dann_data = pd.read_csv('dann_nelec_elec_weired.csv')

In [3]:
dann_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21370 entries, 0 to 21369
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   TEMP        21370 non-null  float64
 1   WS          21370 non-null  float64
 2   WD          21370 non-null  int64  
 3   HUM         21370 non-null  float64
 4   AP          21370 non-null  float64
 5   SLP         21370 non-null  float64
 6   VISIBILITY  21370 non-null  int64  
 7   GTEMP       21370 non-null  float64
 8   N_ELEC      21370 non-null  float64
 9   ELEC        21370 non-null  float64
dtypes: float64(8), int64(2)
memory usage: 1.6 MB


##  For Scaling

In [4]:
X = dann_data.iloc[:,:-2]
Y = dann_data.iloc[:,-2:]

scaler = MinMaxScaler()
scaler.fit(X)
scaled_x = scaler.transform(X)

new_x = pd.DataFrame(scaled_x, index=X.index, columns=X.columns)
new_dann_data = pd.concat([new_x, Y], axis=1)

In [5]:
data_input = new_dann_data.iloc[:,:-2]
source_nelec = new_dann_data[['N_ELEC']]
target_elec = new_dann_data[['ELEC']]

### Source vs Target (Train_Test_Split)

In [6]:
split_for_data_input = int(len(data_input)*0.7)
data_input_trainX = data_input[:split_for_data_input]
data_input_testX = data_input[split_for_data_input:]

split_for_source_nelec = int(len(source_nelec)*0.7)
source_nelec_trainX = source_nelec[:split_for_source_nelec]
source_nelec_testX = source_nelec[split_for_source_nelec:]

split_for_target_elec = int(len(target_elec)*0.7)
target_elec_trainX = target_elec[:split_for_target_elec]
target_elec_testX = target_elec[split_for_target_elec:]

### (Rows, Window_Size, Column) 3차원으로 변환

In [7]:
def buildDataSet(input, target, seqLength):
    xdata = []
    ydata = []
    for i in range(len(input) - seqLength):
        tx = input.iloc[i:i+seqLength]
        ty = target.iloc[i+seqLength-1]
        xdata.append(tx)
        ydata.append(ty)
    return np.array(xdata), np.array(ydata)

In [8]:
data_input_trainx, source_nelec_trainx = buildDataSet(data_input_trainX, source_nelec_trainX, 3)
data_input_testx, source_nelec_testx = buildDataSet(data_input_testX, source_nelec_testX, 3)

data_input_trainx, target_elec_trainx = buildDataSet(data_input_trainX, target_elec_trainX, 3)
data_input_testx, target_elec_testx = buildDataSet(data_input_testX, target_elec_testX, 3)

In [9]:
print(data_input_trainx.shape)
print(data_input_testx.shape)
print(source_nelec_trainx.shape)
print(source_nelec_testx.shape)
print(target_elec_trainx.shape)
print(target_elec_testx.shape)

(14955, 3, 8)
(6409, 3, 8)
(14955, 1)
(6409, 1)
(14955, 1)
(6409, 1)


In [10]:
#Prepare Datasets
BATCH_SIZE = 32
source_dataset = tf.data.Dataset.from_tensor_slices((data_input_trainx, source_nelec_trainx)).batch(BATCH_SIZE*2, drop_remainder=True)
#source_testset = tf.data.Dataset.from_tensor_slices((data_input_testx, source_nelec_testx)).batch(BATCH_SIZE*2, drop_remainder=True)
da_dataset = tf.data.Dataset.from_tensor_slices((data_input_trainx, source_nelec_trainx, data_input_trainx, target_elec_trainx)).batch(BATCH_SIZE, drop_remainder=True)
test_dataset = tf.data.Dataset.from_tensor_slices((data_input_testx, target_elec_testx)).batch(BATCH_SIZE*2, drop_remainder=True) #Test Dataset over Target Domain
test_dataset2 = tf.data.Dataset.from_tensor_slices((data_input_trainx, target_elec_trainx)).batch(BATCH_SIZE*2, drop_remainder=True) #Test Dataset over Target (used for training)

### DANN

In [11]:
@tf.custom_gradient
def gradient_reverse(x, lamda=1.0):
    y = tf.identity(x)
    
    def grad(dy):
        return lamda * -dy, None
    
    return y, grad

class GradientReversalLayer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        
    def call(self, x, lamda=1.0):
        return gradient_reverse(x, lamda)
    
class DANN(Model):
    def __init__(self):
        super().__init__()
        
        # Feature Extractor
        self.feature_extractor_layer0 = LSTM(64, activation='swish', return_sequences=True)
        self.feature_extractor_layer1 = Dropout(0.5)
        self.feature_extractor_layer2 = LSTM(64, activation='swish', return_sequences=False)
        
        # Label regression
        self.label_predcitor_layer0 = Dense(64, activation='relu')
        self.label_predcitor_layer1 = Dense(1)
        
        # Domain Predictor
        self.domain_predictor_layer0 = GradientReversalLayer()
        self.domain_predictor_layer1 = Dense(64, activation='relu')
        self.domain_predictor_layer2 = Dense(2)
        
    def call(self, x,train=False, source_train=True, lamda=1.0):
        # Featrue Extractor
        x = self.feature_extractor_layer0(x)
        x = self.feature_extractor_layer1(x, training=train)
        feature = self.feature_extractor_layer2(x)
        
        #feature = tf.reshape(x, [x.shape[0], -1]) ## shape 2차원으로 바꾸는 거
        
        # Label Predictor
        if source_train is True:
            feature_slice = feature
        else:
            feature_slice = tf.slice(feature, [0, 0], [feature.shape[0] // 2, -1])
            
        lp_x = self.label_predcitor_layer0(feature_slice)
        l_logits = self.label_predcitor_layer1(lp_x)

        # Domain Predictor
        if source_train is True:
            return l_logits
        else:
            dp_x = self.domain_predictor_layer0(feature, lamda) #GradientReversalLayer
            dp_x = self.domain_predictor_layer1(dp_x)
            d_logits = self.domain_predictor_layer2(dp_x)
            return l_logits, d_logits

model = DANN()

In [12]:
def label_mae_loss(true_consumption, pred_consumption):
    mae_loss = tf.reduce_mean(tf.keras.losses.MAE(true_consumption, pred_consumption))
    return mae_loss

def domain_accucary(pred_domain, true_domain):
    domain_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred_domain, labels=true_domain))
    return domain_loss

def get_loss(pred_consumption, true_consumption, pred_domain=None, true_domain=None):
    if pred_domain is None:
        return label_mae_loss(true_consumption, pred_consumption)
    else:
        return domain_accucary(pred_domain, true_domain)

In [13]:
model_reg_optimizer = tf.optimizers.Adam()
model_cla_optimizer = tf.optimizers.SGD()

In [14]:
domain_labels = np.vstack([np.tile([1., 0.], [BATCH_SIZE, 1]),
                           np.tile([0., 1.], [BATCH_SIZE, 1])])
domain_labels = domain_labels.astype('float32')

In [15]:
epoch_reg = tf.keras.metrics.MeanAbsoluteError()
epoch_cla = tf.keras.metrics.CategoricalAccuracy()
source_acc = []  # Source Domain Accuracy while Source-only Training
da_acc = []      # Source Domain Accuracy while DA-training
test_acc = []    # Testing Dataset (Target Domain) Accuracy 
test2_acc = []   # Target Domain (used for Training) Accuracy
EPOCH = 1000

In [16]:
@tf.function
def train_step_source(source_x, source_y, lamda=1.0):
    x = source_x
    y = source_y
    
    with tf.GradientTape() as tape:
        output = model(x, train=True, source_train=True, lamda=lamda)
        
        model_loss = get_loss(output, y)
        epoch_reg(output, y)
        
    gradients_mdan = tape.gradient(model_loss, model.trainable_variables)
    model_reg_optimizer.apply_gradients(zip(gradients_mdan, model.trainable_variables))


@tf.function
def train_step_da(source_x, source_y, target_x=None, target_y=None, lamda=1.0):
    cross_domain_x = tf.concat([source_x, target_x], 0)
    
    with tf.GradientTape() as tape:
        output = model(cross_domain_x, train=True, source_train=False, lamda=lamda)
        l_logits, d_logits = output  # Output from the label predictor and domain predictor

        model_loss = get_loss(l_logits, source_y, d_logits, domain_labels)
        epoch_cla(l_logits, source_y)
          
    gradients_mdan = tape.gradient(model_loss, model.trainable_variables)
    model_cla_optimizer.apply_gradients(zip(gradients_mdan, model.trainable_variables))


@tf.function
def test_step(target_x, target_y):
    x = target_x
    y = target_y
    
    output = model(x, train=False, source_train=True)
    epoch_reg(output, y)


def train(train_mode, epochs=EPOCH):
    
    if train_mode == 'source':
        dataset = source_dataset
        train_func = train_step_source
        acc_list = source_acc
        
        for epoch in range(epochs):
            p = float(epoch) / epochs
            lamda = 2 / (1 + np.exp(-100 * p, dtype=np.float32)) - 1
            lamda = lamda.astype('float32')

            for batch in dataset:
                train_func(*batch, lamda=lamda)

            print("Training: Epoch {} :\t Source MAE : {:.3}".format(epoch, epoch_reg.result()), end='  |  ')
            acc_list.append(epoch_reg.result())
            test(train_mode)
            epoch_reg.reset_states()
        
    elif train_mode == 'domain-adaptation':
        dataset = da_dataset
        train_func = train_step_da
        acc_list = da_acc
        
        for epoch in range(epochs):
            p = float(epoch) / epochs
            lamda = 2 / (1 + np.exp(-100 * p, dtype=np.float32)) - 1
            lamda = lamda.astype('float32')

            for batch in dataset:
                train_func(*batch, lamda=lamda)

            print("Training: Epoch {} :\t Source Accuracy : {:.3%}".format(epoch, epoch_cla.result()), end='  |  ')
            acc_list.append(epoch_cla.result())
            test(train_mode)
            epoch_cla.reset_states()

    else:
        raise ValueError("Unknown training Mode")

def test(train_mode):
    epoch_reg.reset_states()
    epoch_cla.reset_states()
    #Testing Dataset (Target Domain)
    if train_mode == 'source':
        for batch in test_dataset:
            test_step(*batch)

        print("Testing MAE : {:.3}".format(epoch_reg.result()))
        test_acc.append(epoch_reg.result())
        epoch_reg.reset_states()
    
    #Target Domain (used for Training)
    elif train_mode == 'domain-adaptation':
        for batch in test_dataset2:
            test_step(*batch)

        print("Target Domain Accuracy : {:.3%}".format(epoch_cla.result()))
        test2_acc.append(epoch_cla.result())
    epoch_reg.reset_states()
    epoch_cla.reset_states()

In [17]:
#Training
train('source', EPOCH)

Training: Epoch 0 :	 Source MAE : 1.9  |  Testing MAE : 6.72
Training: Epoch 1 :	 Source MAE : 0.763  |  Testing MAE : 6.49
Training: Epoch 2 :	 Source MAE : 0.739  |  Testing MAE : 6.31
Training: Epoch 3 :	 Source MAE : 0.73  |  Testing MAE : 6.19
Training: Epoch 4 :	 Source MAE : 0.728  |  Testing MAE : 6.11
Training: Epoch 5 :	 Source MAE : 0.732  |  Testing MAE : 6.02
Training: Epoch 6 :	 Source MAE : 0.732  |  Testing MAE : 5.94
Training: Epoch 7 :	 Source MAE : 0.734  |  Testing MAE : 5.88
Training: Epoch 8 :	 Source MAE : 0.731  |  Testing MAE : 5.84
Training: Epoch 9 :	 Source MAE : 0.733  |  Testing MAE : 5.78
Training: Epoch 10 :	 Source MAE : 0.727  |  Testing MAE : 5.76
Training: Epoch 11 :	 Source MAE : 0.725  |  Testing MAE : 5.74
Training: Epoch 12 :	 Source MAE : 0.726  |  Testing MAE : 5.73
Training: Epoch 13 :	 Source MAE : 0.722  |  Testing MAE : 5.69
Training: Epoch 14 :	 Source MAE : 0.719  |  Testing MAE : 5.68
Training: Epoch 15 :	 Source MAE : 0.716  |  Testing 

In [18]:
# #Training
# train('domain-adaptation', EPOCH)

In [19]:
# #Plot Results
# x_axis = [i for i in range(0, 20)]

# plt.plot(x_axis, da_acc[20:], label="source accuracy")
# plt.plot(x_axis, test_acc[20:], label="testing accuracy")
# plt.plot(x_axis, test2_acc[20:], label="target accuracy")
# plt.legend()